In [1]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import numpy as np

In [2]:
TRAIN = './data/train_images/'
TILES = './data/train_images_tiles_128_16/'
MASKS = './data/train_label_masks/'
OUT_TRAIN = 'train.zip'
sz = 128
N = 16
if not os.path.exists(TILES):
    os.mkdir(TILES)

In [3]:
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'idx':i})
    return result

In [4]:
x_tot, x2_tot = [], []
names = [name[:-10] for name in os.listdir(MASKS)]
for name in tqdm(names):
    img = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[-1]
    tiles = tile(img)
    for t in tiles:
        img, idx = t['img'], t['idx']
        x_tot.append((img / 255).reshape(-1, 3).mean(0))
        x2_tot.append(((img / 255) ** 2).reshape(-1, 3).mean(0)) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        is_written = cv2.imwrite(f'{TILES}{name}_{idx}.png', img)
        if not is_written:
            print('error write to file', f'{TILES}{name}_{idx}.png')

In [5]:
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr ** 2)
print('mean:', img_avr, '| std:', np.sqrt(img_std))

mean: [0.90949707 0.8188697  0.87795304] | std: [0.36357649 0.49984502 0.40477625]
